In [ ]:
# sample code didnt work. got this from
# https://colab.research.google.com/drive/1Tu7xeYM0qBB-RkqLSnYGrIyzb3q7bYo1#scrollTo=KH91tEik9zZ3

!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > / dev / null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
!pip install findspark


In [ ]:
import findspark

findspark.init()

In [ ]:
!wget https: // jdbc.postgresql.org / download / postgresql-42.2.9.jar


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles

url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz'

spark.sparkContext.addFile(url)



In [ ]:

df = spark.read.option('header', 'true').csv(
    SparkFiles.get('amazon_reviews_us_Video_Games_v1_00.tsv.gz'),
    inferSchema=True,
    sep='\t',
    timestampFormat='yyyy-mm-dd'
)

df.show()

Count the number of records (rows) in the dataset.



In [ ]:
df.count()

Transform dataset to fit the schema

In [ ]:
# drop nulls
df = df.dropna()

# see current schema
df.printSchema()

In [ ]:
df_conv = df.withColumn(
    'review_date',
    df['review_date'].cast(DateType())
)

df_conv.printSchema()

In [ ]:
review_cols = [
    'review_id',
    'customer_id',
    'product_id',
    'product_parent',
    'review_date'
]

product_cols = [
    'product_id',
    'product_title'
]

customer_cols = [
    'customer_id',
    'customer_count'  # must be made first
]

vine_cols = [
    'review_id',
    'star_rating',
    'helpful_votes',
    'total_votes',
    'vine'
]

table_cols = [review_cols, product_cols, vine_cols]



In [ ]:
customers = df_conv.groupBy('customer_id').count()
customers.show()
customers.printSchema()

In [ ]:
def make_tables(df, cols):
    output_df = df.select(cols)
    output_df.show()
    output_df.printSchema()

    return output_df


dfs = []

for c in table_cols:
    app_df = make_tables(df_conv, c)
    dfs.append(app_df)

dfs

In [ ]:
review_id_table = dfs[0]
products = dfs[1]


In [ ]:
vine_table = dfs[2]
vine_table = vine_table.where(col('vine') == 'Y').show()

In [ ]:

mode = 'append'
jdbc_url = "jdbc:postgresql://<endpoint>:5432/<database name>"

config = {
    "user": '<username>',
    "password": '<password>',
    "driver": "org.postgresql.Driver"
}

review_id_table.write.jdbc(
    url=jdbc_url,
    table='review_id_table',
    mode=mode,
    properties=config
)

products.write.jdbc(
    url=jdbc_url,
    table='products',
    mode=mode,
    properties=config
)

vine_table.write.jdbc(
    url=jdbc_url,
    table='vine_table',
    mode=mode,
    properties=config
)

customers.write.jdbc(
    url=jdbc_url,
    table='customers',
    mode=mode,
    properties=config
)